In [5]:
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append("/content/drive/MyDrive/Colab Notebooks/instquality/")

import os
os.chdir("/content/drive/MyDrive/Colab Notebooks/instquality/")

import warnings
warnings.filterwarnings("ignore")

Mounted at /content/drive


In [9]:
import os, time
import numpy as np
import pandas as pd
from scipy.stats import norm
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from IPython.display import display
from joblib import Parallel, delayed

def _yearwise_splits(years, min_train_years=8, stride=2):
    ys = np.asarray(years)
    uniq = np.sort(np.unique(ys))
    for k in range(min_train_years, len(uniq), stride):
        train = np.where(np.isin(ys, uniq[:k]))[0]
        val = np.where(ys == uniq[k])[0]
        yield train, val

def _rmse_for_param(param, X, y, years, model_type='lasso', l1_ratio=0.5, min_train_years=8, stride=2):
    rmse = []
    if model_type == 'lasso':
        model = Lasso(alpha=param, max_iter=10000, warm_start=True)
    elif model_type == 'ridge':
        model = Ridge(alpha=param)
    elif model_type == 'elastic':
        model = ElasticNet(alpha=param, l1_ratio=l1_ratio, max_iter=10000)
    else:
        raise ValueError(f"Unknown model_type: {model_type}")
    for tr, va in _yearwise_splits(years, min_train_years, stride):
        model.fit(X.iloc[tr], y.iloc[tr])
        pred = model.predict(X.iloc[va])
        rmse.append(np.sqrt(mean_squared_error(y.iloc[va], pred)))
    return param, float(np.mean(rmse)) if rmse else np.inf

def choose_param_expanding(X, y, years, model_type='lasso', l1_ratio=0.5, params_coarse=None, params_fine=12, min_train_years=8, stride=2, n_jobs=-1):
    if params_coarse is None:
        params_coarse = np.logspace(0, 5, 20) if model_type == 'ridge' else np.logspace(-4, 1, 15)
    coarse = Parallel(n_jobs=n_jobs)(
        delayed(_rmse_for_param)(p, X, y, years, model_type, l1_ratio, min_train_years, stride)
        for p in sorted(params_coarse, reverse=True)
    )
    p0 = min(coarse, key=lambda t: t[1])[0]
    low, high = p0/10, p0*10
    fine_grid = np.logspace(np.log10(low), np.log10(high), params_fine)
    fine = Parallel(n_jobs=n_jobs)(
        delayed(_rmse_for_param)(p, X, y, years, model_type, l1_ratio, min_train_years, stride)
        for p in sorted(fine_grid, reverse=True)
    )
    return min(fine, key=lambda t: t[1])[0]

def _dm_test(e0, e1):
    d = e0**2 - e1**2
    T = len(d)
    v = np.var(d, ddof=1)
    if T < 2 or v == 0 or np.isnan(v):
        return np.nan, np.nan
    stat = d.mean() / np.sqrt(v / T)
    pval = 2 * (1 - norm.cdf(abs(stat)))
    return stat, pval

def decompose_r2(y_true, y_pred, X, coefs, feature_names):
    y_true = np.asarray(y_true).flatten()
    y_pred = np.asarray(y_pred).flatten()
    X = np.asarray(X)
    coefs = np.asarray(coefs).flatten()
    y_centered = y_true - y_true.mean()
    X_centered = X - X.mean(axis=0)
    contributions = []
    for j in range(X.shape[1]):
        cov_j = np.dot(X_centered[:, j], y_centered) / len(y_centered)
        contrib = coefs[j] * cov_j
        contributions.append(contrib)
    contributions = np.array(contributions)
    total_contrib = contributions.sum()
    r2 = r2_score(y_true, y_pred)
    if total_contrib != 0:
        normalized_contrib = (contributions / total_contrib) * r2
    else:
        normalized_contrib = np.zeros_like(contributions)
    decomp_df = pd.DataFrame({
        'variable': feature_names,
        'coefficient': coefs,
        'contribution': contributions,
        'r2_contribution': normalized_contrib,
        'r2_contribution_pct': normalized_contrib * 100
    })
    decomp_df = decomp_df.reindex(decomp_df['r2_contribution'].abs().sort_values(ascending=False).index)
    return decomp_df

def run_unified_regularized_regression(
    name_prefix="base",
    data_path="saved/df.dat",
    macro_cols=None,
    iq_cols=None,
    y_name="tgt_spread_lag",
    spread_col="tgt_spread",
    year_col="year",
    split_year=2015,
    model_types=['lasso', 'ridge', 'elastic'],
    agnostic=False,
    param_factors=[0.5, 1.0, 2.0],
    l1_ratio=0.5,
    use_expanding_cv=True,
    min_train_years=8,
    cv_stride=2,
    r2_benchmark=None,
    rmse_benchmark=None,
    save_results=True,
    output_dir="specs",
    n_jobs=-1
):
    print(f"\n{'='*80}")
    print(f"UNIFIED REGULARIZED REGRESSION: {name_prefix.upper()}")
    print(f"Specification: {'Agnostic (β unrestricted)' if agnostic else 'β=1 (mean reversion)'}")
    print(f"Models: {', '.join([m.upper() for m in model_types])}")
    print(f"{'='*80}\n")
    start_time = time.time()

    df = pd.read_pickle(data_path)
    df[year_col] = df[year_col].astype(int)

    if macro_cols is None:
        macro_cols = []
    if iq_cols is None:
        iq_cols = []

    train_data = df[df[year_col] <= split_year].copy()
    test_data = df[df[year_col] > split_year].copy()

    print(f"Train: {train_data[year_col].min()}-{train_data[year_col].max()} ({len(train_data)} obs)")
    print(f"Test:  {test_data[year_col].min()}-{test_data[year_col].max()} ({len(test_data)} obs)\n")

    feature_cols = macro_cols + iq_cols
    if spread_col in feature_cols and not agnostic:
        feature_cols = [c for c in feature_cols if c != spread_col]

    X_train_raw = train_data[feature_cols].copy()
    X_test_raw = test_data[feature_cols].copy()
    y_train = train_data[y_name].copy()
    y_test = test_data[y_name].copy()
    years_train = train_data[year_col].values

    if not agnostic:
        if spread_col not in train_data.columns:
            raise ValueError(f"β=1 specification requires '{spread_col}' column")
        spread_train = train_data[spread_col].copy()
        spread_test = test_data[spread_col].copy()
        mask_train = spread_train.notna() & y_train.notna()
        mask_test = spread_test.notna() & y_test.notna()
        X_train_raw = X_train_raw.loc[mask_train]
        X_test_raw = X_test_raw.loc[mask_test]
        y_train = y_train.loc[mask_train]
        y_test = y_test.loc[mask_test]
        spread_train = spread_train.loc[mask_train]
        spread_test = spread_test.loc[mask_test]
        years_train = years_train[mask_train.values]
        y_train_adj = y_train - spread_train
        y_test_adj = y_test - spread_test
        print(f"β=1 specification: predicting (y_t+1 - spread_t)")
        print(f"After removing NaN: {len(y_train_adj)} train, {len(y_test_adj)} test obs\n")
    else:
        y_train_adj = y_train
        y_test_adj = y_test
        spread_train = None
        spread_test = None

    scaler = StandardScaler()
    imputer = SimpleImputer(strategy='median')
    X_train_imp = imputer.fit_transform(X_train_raw)
    X_test_imp = imputer.transform(X_test_raw)
    X_train = pd.DataFrame(scaler.fit_transform(X_train_imp), index=X_train_raw.index, columns=X_train_raw.columns)
    X_test = pd.DataFrame(scaler.transform(X_test_imp), index=X_test_raw.index, columns=X_test_raw.columns)
    feature_names = list(X_train.columns)
    print(f"Features: {len(feature_names)}\n")

    all_results = {}

    for model_type in model_types:
        print(f"\n{'-'*80}")
        print(f"RUNNING {model_type.upper()}")
        print(f"{'-'*80}\n")

        if use_expanding_cv:
            print("Selecting optimal parameter via expanding window CV...")
            optimal_param = choose_param_expanding(X_train, y_train_adj, years_train, model_type=model_type, l1_ratio=l1_ratio, min_train_years=min_train_years, stride=cv_stride, n_jobs=n_jobs)
        else:
            params = np.logspace(0, 5, 30) if model_type == 'ridge' else np.logspace(-4, 1, 30)
            best_score = -np.inf
            optimal_param = params[0]
            for p in params:
                if model_type == 'lasso':
                    m = Lasso(alpha=p, max_iter=10000)
                elif model_type == 'ridge':
                    m = Ridge(alpha=p)
                else:
                    m = ElasticNet(alpha=p, l1_ratio=l1_ratio, max_iter=10000)
                m.fit(X_train, y_train_adj)
                score = m.score(X_train, y_train_adj)
                if score > best_score:
                    best_score = score
                    optimal_param = p

        print(f"Optimal parameter: {optimal_param:.6f}\n")

        results_rows = []
        models = {}

        for factor in param_factors:
            param = optimal_param * factor
            if model_type == 'lasso':
                model = Lasso(alpha=param, max_iter=10000)
            elif model_type == 'ridge':
                model = Ridge(alpha=param)
            else:
                model = ElasticNet(alpha=param, l1_ratio=l1_ratio, max_iter=10000)
            model.fit(X_train, y_train_adj)
            y_pred_test_adj = model.predict(X_test)
            y_pred_train_adj = model.predict(X_train)
            if not agnostic:
                y_pred_test = spread_test + y_pred_test_adj
                y_pred_train = spread_train + y_pred_train_adj
            else:
                y_pred_test = y_pred_test_adj
                y_pred_train = y_pred_train_adj
            r2_train = r2_score(y_train, y_pred_train)
            r2_test = r2_score(y_test, y_pred_test)
            rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
            active_vars = np.sum(model.coef_ != 0)
            if not agnostic:
                e_benchmark = y_test - spread_test
                e_model = y_test - y_pred_test
                dm_stat, dm_p = _dm_test(e_benchmark, e_model)
            else:
                dm_stat, dm_p = np.nan, np.nan
            results_rows.append({'factor': factor, 'param': param, 'R²_train': r2_train, 'R²_test': r2_test, 'RMSE_test': rmse_test, 'Active_vars': active_vars, 'DM_stat': dm_stat, 'DM_p': dm_p})
            models[factor] = (model, y_pred_test)

        results_df = pd.DataFrame(results_rows)
        display(results_df)

        best_idx = results_df['R²_test'].idxmax()
        best_row = results_df.iloc[best_idx]
        best_model, best_pred = models[best_row['factor']]

        print(f"\nBest {model_type.upper()}:")
        print(f"  Parameter: {best_row['param']:.6f}")
        print(f"  R² (test): {best_row['R²_test']:.4f}")
        print(f"  RMSE (test): {best_row['RMSE_test']:.4f}")
        print(f"  Active vars: {int(best_row['Active_vars'])}")
        if r2_benchmark is not None:
            print(f"  ΔR² vs benchmark: {(best_row['R²_test'] - r2_benchmark)*100:+.2f} p.p.")
        if rmse_benchmark is not None:
            print(f"  ΔRMSE vs benchmark: {(best_row['RMSE_test'] - rmse_benchmark):+.4f}")
        if not agnostic:
            print(f"  DM test: stat={best_row['DM_stat']:.2f}, p={best_row['DM_p']:.3f}")

        print(f"\n{'='*80}")
        print(f"VARIABLE IMPORTANCE DECOMPOSITION ({model_type.upper()})")
        print(f"{'='*80}\n")

        decomp_df = decompose_r2(y_test, best_pred, X_test, best_model.coef_, feature_names)
        top_n = min(20, len(decomp_df))
        print(f"Top {top_n} contributors to R²:\n")
        display(decomp_df.head(top_n)[['variable', 'coefficient', 'r2_contribution', 'r2_contribution_pct']])

        total_r2_explained = decomp_df['r2_contribution'].sum()
        print(f"\nTotal R² explained: {total_r2_explained:.4f}")
        print(f"Sum of absolute contributions: {decomp_df['r2_contribution'].abs().sum():.4f}")

        if save_results:
            os.makedirs(output_dir, exist_ok=True)
            suffix = f"{'_agn' if agnostic else ''}"
            results_df.to_pickle(f"{output_dir}/{model_type}_{name_prefix}{suffix}_results.dat")
            decomp_df.to_pickle(f"{output_dir}/{model_type}_{name_prefix}{suffix}_decomp.dat")
            pd.Series(best_model.coef_, index=feature_names).to_pickle(f"{output_dir}/{model_type}_{name_prefix}{suffix}_coefs.dat")

        all_results[model_type] = {'results_df': results_df, 'best_model': best_model, 'best_row': best_row, 'decomposition': decomp_df, 'predictions': best_pred}

    elapsed = time.time() - start_time
    print(f"\n{'='*80}")
    print(f"COMPLETED IN {elapsed:.1f}s")
    print(f"{'='*80}\n")
    return all_results

In [ ]:
# Define your columns
df = pd.read_pickle("saved/df.dat")
id_cols = ["country","year","iso_code_1","iso_code_2","region"]
exclude = id_cols + [c for c in df.columns if c.startswith("tgt_")]
macro_cols = [c for c in df.columns if c.startswith("wb_") and not c.startswith("wb_iq_") and c not in exclude]
iq_cols = [c for c in df.columns if (c.startswith("wb_iq_") or (not c.startswith("wb_") and c not in exclude))]

# Run specification
results = run_unified_regularized_regression(
    name_prefix="base",
    data_path="saved/df.dat",
    macro_cols=macro_cols,
    iq_cols=iq_cols,
    y_name='tgt_spread',
    spread_col="tgt_spread", # only used for next-year spread prediction, especially in non-agnostic version
    year_col="year",
    split_year=2015,
    model_types=['lasso', 'ridge', 'elastic'],
    agnostic=True,
    param_factors=np.logspace(-10, 10, 25),
    l1_ratio=0.5,
    use_expanding_cv=True,
    min_train_years=8,
    cv_stride=2,
    r2_benchmark=None,
    rmse_benchmark=None,
    save_results=True,
    output_dir="specs",
    n_jobs=-1
)


UNIFIED REGULARIZED REGRESSION: BASE
Specification: Agnostic (β unrestricted)
Models: LASSO, RIDGE, ELASTIC

Train: 1960-2015 (1652 obs)
Test:  2016-2024 (541 obs)

Features: 245


--------------------------------------------------------------------------------
RUNNING LASSO
--------------------------------------------------------------------------------

Selecting optimal parameter via expanding window CV...
